Github code from authors stored in picrispr-main

TODO:

1. Preprocess data (dir is offtarget_260520_nuc.csv.zip (crisprsql dataset)).
we can use the code from load_data.py and encoding.py.
the author calls the functions from those classes in picrispr.py.

2. Run models (models.py)

3. Test model (test_input.csv)

In [1]:
#Load files
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/.shortcut-targets-by-id/1xWx4iaOa4dZLKNbO8SfeXGyQyzJoKG1L/piCrispr2.0/picrispr-main
!ls

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1xWx4iaOa4dZLKNbO8SfeXGyQyzJoKG1L/piCrispr2.0/picrispr-main
 default_vals   nontransformed.png	      'picrispr data format.gsheet'   test_input.csv
 encoding.py    offtarget_260520_nuc.csv.zip   picrispr.py		      transformedback.png
 load_data.py  'output(1).gsheet'	       __pycache__		     'WillB (1).py'
 models         output.csv		       README.md		      WillB.py
 models.py      output_Option3RNN.csv	       requirements.txt


In [2]:


#!python WillB.py test_input.csv 3
!python picrispr.py test_input.csv 2 models False False
#picrispr.py 588 throw no such file or directory when running torch models
#Searches for 'models/trainresult_torch_interface_type_s2_class.pickle'
#Directory has 'models/trainresult_tf_interface_type_s2_class.pickle'
#Forcing torch version to look for tf file instead gives error... (screenshot2) and (screenshot4) if mode is changed in load function


tf _s4 __________________________________
2it [00:00, 10292.77it/s]
839it [00:00, 17828.30it/s]
2it [00:00, 9619.96it/s]
61it [00:00, 17731.83it/s]
74it [00:00, 28129.28it/s]
22it [00:00, 25890.77it/s]
/content/gdrive/.shortcut-targets-by-id/1xWx4iaOa4dZLKNbO8SfeXGyQyzJoKG1L/piCrispr2.0/picrispr-main/picrispr.py:593: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:605.)
  return sparse_tensortype(indices, values, x.size())
using GPU device Tesla T4
done
config[mode] is  tf  so dm.mode was set to be  tf
loading model...load function was passed  tf  for mode
done
preparing dataset...done
obtaining predictions...
32/32 [==============================] - 10s 309ms/step
successfully saved predictions to output.csv


In [3]:
!ls
#load data preproces here

 default_vals   nontransformed.png	      'picrispr data format.gsheet'   test_input.csv
 encoding.py    offtarget_260520_nuc.csv.zip   picrispr.py		      transformedback.png
 load_data.py  'output(1).gsheet'	       __pycache__		     'WillB (1).py'
 models         output.csv		       README.md		      WillB.py
 models.py      output_Option3RNN.csv	       requirements.txt


In [4]:
import pandas as pd
filePath = "offtarget_260520_nuc.csv.zip"
df = pd.read_csv(filePath, low_memory=False)
print(df.columns)

Index(['id', 'experiment_id', 'target_chr', 'target_start', 'target_end',
       'target_strand', 'target_sequence', 'target_context', 'target_geneid',
       'grna_target_id', 'grna_target_chr', 'grna_target_start',
       'grna_target_end', 'grna_target_strand', 'grna_target_sequence',
       'genome', 'cell_line', 'measured', 'cleavage_freq', 'epigenetics_ids',
       'epigen_ctcf', 'epigen_dnase', 'epigen_rrbs', 'epigen_h3k4me3',
       'epigen_drip', 'energy_1', 'energy_2', 'energy_3', 'energy_4',
       'energy_5', 'GCContent', 'WSScore', 'YRScore', 'NucleotideBDM',
       'StrongWeakBDM', 'NuPoP_Occup_147_human', 'NuPoP_Viterbi_147_human',
       'NuPoP_Affinity_147_human', 'nuCpos_Occup_147_yeast',
       'nuCpos_Viterbi_147_yeast', 'nuCpos_Affinity_147_yeast',
       'VanDerHeijden', 'LeNupH3Q85C', 'mismatch_num',
       'energy_4*(energy_3/energy_2)', 'energy_2-energy_4*(energy_3/energy_2)',
       'energy_1-energy_4*(energy_3/energy_2)'],
      dtype='object')


In [5]:

import torch
from torch.utils.data import Dataset
import torch.nn.functional as F

class PaddedDataset(Dataset):
    def __init__(self, original_dataset, desired_length):
        self.original_dataset = original_dataset
        self.desired_length = desired_length

    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        original_tensor, label, extra = self.original_dataset[idx]

        # Calculate the amount of padding needed on the right side
        padding_right = self.desired_length - len(original_tensor)

        # Use F.pad to pad the tensor only on the right side
        padded_tensor = F.pad(original_tensor, (0, padding_right), 'constant', value=0)

        return (padded_tensor, label, extra)

# Example usage
original_dataset = [torch.tensor([1, 2, 3]), torch.tensor([4, 5]), torch.tensor([6, 7, 8, 9])]
desired_length = 144

In [10]:
import sys
from encoding import oneHotSingleNuclTargetMismatchType, oneHotSingleNuclTargetMismatch, oneHotSingleNucl, normaliseCF
from models import mySequential, vecToMatEncoder, vecToMatEncoding
import pickle
from tqdm import tqdm
import xgboost as xgb
import torch
import tensorflow as tf
from scipy.stats import spearmanr
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from picrispr import CSVDataset


use_gpu = True

filenames = ["xgboost_interface_type_epi", "torch_eng", "torch_engnuc", "torch_interface_type_nuc", "tf_eng", "tf_engnuc"]
modelnames =["XGB_S3E2", "CNN_S2E0", "CNN_S4E0", "CNN_S5E2", "RNN_S2E3", "RNN_S4E3"]

# define models
seq_features = "target_sequence, grna_target_sequence, "
energy_features = "energy_2+energy_1-(energy_3*energy_4/energy_2), "
epigen_features = "epigen_ctcf, epigen_dnase, epigen_rrbs, epigen_h3k4me3, epigen_drip, "
nuc_features = "NucleotideBDM, NuPoP_Affinity_147_human, GCContent, "
numBpWise = 3

seq_energy_feat         = seq_features + energy_features + "experiment_id"
seq_energy_epi_feat     = seq_features + energy_features + epigen_features + "experiment_id"
seq_energy_nuc_feat     = seq_features + nuc_features + energy_features + "experiment_id"
seq_energy_epi_nuc_feat = seq_features + nuc_features + energy_features + epigen_features + "experiment_id"

# retrieve user input
filePath = "offtarget_260520_nuc.csv.zip"
modelNum   = 3
home       = "models"
regression = False
replace_all_nans = False

args = [("", home, "torch", False, regression, "s4", False), # E3+CRISPRNetStyle, CNN, s4
        ("", home, "torch", False, regression, "s2", False), # E3+CRISPRNetStyle, CNN, s2
        ("", home, "tf",    False, regression, "s4", False), # E3+CRISPRNetStyle, RNN, s4
        ("", home, "tf",    False, regression, "s2", False), # E3+CRISPRNetStyle, RNN, s2
        ("", home, "torch", True,  regression, "s4", True),  # E2,                CNN, s4
        ("", home, "torch", True,  regression, "s2", True),  # E2,                CNN, s2
        ("", home, "tf",    True,  regression, "s4", True),  # E2,                RNN, s4
        ("", home, "tf",    True,  regression, "s2", True),  # E2,                RNN, s2
        ]

kwargs = [{'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': True},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': True},
          {'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': True},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': True},
          {'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': False},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': False},
          {'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': False},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': False}
          ]

# load user dataset and chosen model
dataset = CSVDataset(filePath)

config = {"dbFields": kwargs[modelNum]['dbFields'],
          "numBpWise": kwargs[modelNum]['numBpWise'],
          "mode": args[modelNum][2],
          "mismatchType": args[modelNum][3],
          "interfaceMode": args[modelNum][6],
          "chooseSpecies": ["hg19", "hg38"],
          "regression": args[modelNum][4],
          "CRISPRNetStyle": kwargs[modelNum]['CRISPRNetStyle'],
          }

print(config)

filenameAppendix  = "_interface" if config["interfaceMode"] else ""
filenameAppendix += "_type"      if config["mismatchType"] and config["interfaceMode"]  else ""
filenameAppendix += "_"+args[modelNum][5]

print(config["mode"], filenameAppendix, "__________________________________")

if config["mismatchType"]:
    oneHotFct = oneHotSingleNuclTargetMismatchType
    featurenames = ['A_match', 'T_match', 'C_match', 'G_match',
                    'A_mismT', 'T_mismC', 'C_mismG', 'G_mismA',
                    'A_mismC', 'T_mismG', 'C_mismA', 'G_mismT',
                    'A_mismG', 'T_mismA', 'C_mismT', 'G_mismC']
elif config["interfaceMode"]:
    oneHotFct = oneHotSingleNuclTargetMismatch
    featurenames = ['A', 'A_mism', 'T', 'T_mism', 'C', 'C_mism', 'G', 'G_mism']
else:
    oneHotFct = oneHotSingleNucl
    featurenames = ['A',           'T',           'C',           'G']

featurenames.extend(" ".join(config["dbFields"].split()).split(', ')[2:]) # append whatever database fields apart from guide and target sequence are used


# get data matrix from user input
filenameAppendix += "_class" if not config["regression"] else ""

print("encoding dataset...", end = '')
dm = dataset.getDataMatrix(config["dbFields"], oneHotFct, normaliseCF, chooseSpecies=config["chooseSpecies"], filenameAppendix=filenameAppendix,
                            featurenames=featurenames, mode=config["mode"], numBpWise=config["numBpWise"], test_size=0.2, doSplit=True,
                            CRISPRNetStyle=config["CRISPRNetStyle"], replace_all_nans=replace_all_nans)
print("done")

print("config[mode] is ", config["mode"], " so dm.mode was set to be ", dm.mode)
dm.mode = config["mode"]
dm.interfaceMode = config["interfaceMode"]
dm.mismatchType = config["mismatchType"]
dm.regression = config["regression"]
dm.CRISPRNetStyle = config["CRISPRNetStyle"]

# load model
print("loading model...", end = '')
#result = TrainResult.load(home, config["mode"], filenameAppendix, device="gpu:0" if use_gpu else "cpu")
print("done")

# predict on ext set
isHPC = True
#bs = int(7e4) if isHPC or dm.mode != "torch" else 35000
bs = 1024
# don't use experiment_id column
dm.toDense()
dm = dm.dropColumn(-1)

print("preparing dataset...", end = '')
trainDataset, validDataset, extDataset = dm.prepareDataset(cutoff_class=-4, addGaussian=False)
#trainDataset = PaddedDataset(trainDataset, desired_length)
#validDataset = PaddedDataset(validDataset, desired_length)
#extDataset = PaddedDataset(extDataset, desired_length)
#dm.prepareDataloaders(trainDataset, validDataset, extDataset, bs, balanceClasses=False, ignoreExtSet=True, doSampling=False)
#print("done")

{'dbFields': 'target_sequence, grna_target_sequence, energy_2+energy_1-(energy_3*energy_4/energy_2), experiment_id', 'numBpWise': 0, 'mode': 'tf', 'mismatchType': False, 'interfaceMode': False, 'chooseSpecies': ['hg19', 'hg38'], 'regression': False, 'CRISPRNetStyle': True}
tf _s2 __________________________________
encoding dataset...

103it [00:00, 17093.87it/s]
1505it [00:00, 18819.14it/s]
78it [00:00, 20592.67it/s]
3801it [00:00, 21921.69it/s]
51it [00:00, 13198.59it/s]
5413it [00:00, 18399.54it/s]
549it [00:00, 18412.10it/s]
44624it [00:01, 23288.72it/s]
72it [00:00, 20197.29it/s]
29327it [00:01, 18413.01it/s]
7374it [00:00, 17581.87it/s]
41429it [00:01, 21546.08it/s]
866it [00:00, 27742.27it/s]
6386it [00:00, 17792.45it/s]
231it [00:00, 19238.40it/s]
27995it [00:01, 16373.30it/s]
11it [00:00, 8965.67it/s]
2380it [00:00, 15185.10it/s]
86it [00:00, 11168.19it/s]
2438it [00:00, 17097.05it/s]
108it [00:00, 18931.16it/s]
3258it [00:00, 17326.41it/s]
61it [00:00, 12371.38it/s]
1747it [00:00, 17972.34it/s]
272it [00:00, 16672.52it/s]
23463it [00:01, 16951.73it/s]
203it [00:00, 17329.01it/s]
12373it [00:00, 21741.98it/s]
23it [00:00, 16921.42it/s]
21519it [00:00, 22298.63it/s]
11213it [00:00, 19603.10it/s]
2895it [00:00, 18297.47it/s]


using GPU device Tesla T4
done
config[mode] is  tf  so dm.mode was set to be  tf
loading model...done
preparing dataset...

In [15]:
l0 = []
l1 = []

for batch in trainDataset:
    input, target, extra = batch
    if target.item() == 0:
      l0.append((input, target))
    else:
      l1.append((input, target))


In [23]:
print(len(l0))
print(len(l1))

17005
17005


In [33]:
import numpy as np

l0 = np.array(l0)
l1 = np.array(l1)
np.random.shuffle(l0)

l0 = l0[:len(l1)]

l = np.concatenate((l0, l1))

np.random.shuffle(l)

In [ ]:
print(l)

In [35]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)
    def __getitem__(self, idx):
        input_sample = torch.tensor(self.inputs[idx])
        target_sample = torch.tensor(self.targets[idx])

        sample = (input_sample, target_sample)

        return sample

In [36]:
trainDataset = MyDataset(l[:,0], l[:,1])

In [37]:
from torch.utils.data import DataLoader

traindataloader = DataLoader(trainDataset, batch_size=bs, shuffle=True)
testdataloader = DataLoader(validDataset, batch_size=bs, shuffle=True)

In [38]:
print(trainDataset[0])

(tensor([ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,
         0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
         1.0000,  0.0000,  0.0000,  0.0

<ipython-input-35-e968f9abd1eb>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_sample = torch.tensor(self.inputs[idx])
<ipython-input-35-e968f9abd1eb>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_sample = torch.tensor(self.targets[idx])


In [ ]:
1-0.9155954378058586

0.08440456219414139

In [ ]:
sum([1.0921854333355017, 11.847700811478305])

12.939886244813806

In [ ]:
[1.0921854333355017/12.939886244813806, 11.847700811478305/12.939886244813806]

[0.0844045621941414, 0.9155954378058586]

TwoLayerNet: 91.56% Accuracy

In [ ]:
from models import TwoLayerNet, ThreeLayerNet, LinearRegressionModel, ConvolutionalNet
import torch.nn as nn
import torch.optim as optim
import numpy as np

lossf = nn.MSELoss()
#model = ConvolutionalNet(False)
#model = ThreeLayerNet(139, 256, 1)
model = TwoLayerNet(139, 256, 1)
#model = LinearRegressionModel(139, 256) RuntimeError: shape '[1, 60000]' is invalid for input of size 15360000

encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

model = nn.Sequential(
    nn.Linear(139,512),
    transformer_encoder,
    nn.Linear(512, 1),
    nn.Sigmoid()
)

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):

    model.train()
    for batch in traindataloader:
        inputs, targets, extra = batch
        # Forward pass

        outputs = 8*model(inputs)-4

        print(outputs)

        # Compute loss
        loss = lossf(outputs, targets)
        print(loss.item())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}')

    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_samples = 0

    val_loss = 0
    batches = 0

    with torch.no_grad():  # Disable gradient computation during validation
        for batch in testdataloader:
            inputs, targets, extra = batch
            outputs = 8*model(inputs)-4
            loss = lossf(outputs, targets)
            '''outputs = F.softmax(outputs, dim=1)

            pred = outputs.numpy()
            labels = targets.numpy()

            # Assuming labels are in the range [0, 1] (0 for class 0, 1 for class 1)
            #labels = labels.float().view(-1, 1)  # Ensure labels are of type float and have shape (batch_size, 1)

            # Compute accuracy
            predictions = pred[:,1] > 0.5  # Threshold at 0.5 for binary classification

            correct = np.sum(predictions == labels.astype(int))
            total_correct += correct
            total_samples += len(labels)'''

            val_loss += loss.item()
            batches += 1

    #accuracy = total_correct / total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Test Loss: {val_loss/batches:.4f}')



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


tensor([[1.2803],
        [1.1454],
        [1.3796],
        ...,
        [1.5626],
        [1.3387],
        [1.0170]], grad_fn=<SubBackward0>)
25.347898483276367


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([1024, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([[-3.9987],
        [-3.9987],
        [-3.9989],
        ...,
        [-3.9986],
        [-3.9987],
        [-3.9987]], grad_fn=<SubBackward0>)
1.6533911228179932
tensor([[-3.9982],
        [-3.9985],
        [-3.9984],
        ...,
        [-3.9983],
        [-3.9985],
        [-3.9983]], grad_fn=<SubBackward0>)
1.4640421867370605
tensor([[-3.9916],
        [-3.9940],
        [-3.9921],
        ...,
        [-3.9924],
        [-3.9920],
        [-3.9937]], grad_fn=<SubBackward0>)
1.7684693336486816
tensor([[-3.9288],
        [-3.9310],
        [-3.9291],
        ...,
        [-3.9266],
        [-3.9312],
        [-3.9270]], grad_fn=<SubBackward0>)
1.4203565120697021
tensor([[-3.3925],
        [-2.9330],
        [-3.3282],
        ...,
        [-2.9613],
        [-2.9745],
        [-3.2781]], grad_fn=<SubBackward0>)
1.8699829578399658
tensor([[-3.9925],
        [-3.9930],
        [-3.9912],
        ...,
        [-3.9908],
        [-3.9936],
        [-3.9911]], grad_fn=<SubBackw

KeyboardInterrupt: ignored